# ELYZA-tasks-100 による性能評価

In [ ]:
#!pip install accelerate autoawq datasets jsonlines sentencepiece
!pip install accelerate datasets jsonlines sentencepiece
# https://github.com/oobabooga/text-generation-webui/issues/4517#issuecomment-1815279467
!pip install https://github.com/casper-hansen/AutoAWQ/releases/download/v0.1.7/autoawq-0.1.7+cu118-cp310-cp310-linux_x86_64.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.0/154.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.0/229.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

## ELYZA-tasks-100 データセットのダウンロード

モデルのダウンロードには 🤗datasets ライブラリを用いる。

データは `/content/dataset.jsonl` に保存する。

In [ ]:
import json
from pathlib import Path

from datasets import load_dataset

# Load the dataset
ds = load_dataset("elyza/ELYZA-tasks-100")

# Function to convert dataset to JSONL format and print
def dataset_to_jsonl(dataset, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        for entry in dataset:
            # Construct JSON object
            json_obj = {
                "input_text": entry['input'],
                "output_text": entry['output'],
                "eval_aspect": entry['eval_aspect']
            }

            # Write JSON object to file in JSONL format
            json_str = json.dumps(json_obj, ensure_ascii=False)
            file.write(json_str + '\n')

# Convert and write the dataset to a file in JSONL format
path_jsonl = Path("/content/dataset.jsonl")

if (path_jsonl.parent is not None) and (not path_jsonl.parent.exists()):
    path_jsonl.parent.mkdir(parents=True, exist_ok=True)

dataset_to_jsonl(ds["test"], path_jsonl)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

## 評価

ELYZA-tasks-100 データセットを用いて、指定したモデルの評価を実行する。

### プロンプトについて

モデルに回答を生成させるときのプロンプトは、各モデルのモデルカード (🤗HuggingFace の README ページ) に記載されているプロンプトを使用する。

> 📒 **ノート**
>
> この方法は、質問を直接入力する場合や、全モデルで共通のプロンプトを用いる場合に比べて公平性が劣る。
> しかし、実利用時は各モデルが最もよい性能を発揮するように、各モデルのプロンプトをチューニングすることが想定される。
> モデルカード記載のプロンプトをチューニング済みのプロンプトと見なすことで、実利用時の性能を比較することを狙いとする。

### ハイパーパラメータについて

ハイパーパラメータは 🤗HuggingFace の Code Snippet のとおりとする（公開者が指定したハイパーパラメータを、チューン済みとみなす）。
ただし、長文の回答を要求する設問もあるため、`max_new_tokens` は `1024` に統一する。

In [ ]:
import json
import jsonlines
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Setup model and tokenizer
model_name = "augmxnt/shisa-7b-v1"
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=True,
    device_map="auto")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto",
)

def generate_text(input_text):
    # ✋プロンプトは、モデルに合わせて調整（HuggingFace のモデルカード参照）
    # Ref: https://huggingface.co/augmxnt/shisa-7b-v1#usage
    prompt = "あなたは役に立つアシスタントです。"
    chat = [{"role": "system", "content": prompt}]
    chat.append({"role": "user", "content": input_text})

    # Generate - add_generation_prompt to make sure it continues as assistant
    formatted_prompt = tokenizer.apply_chat_template(chat, add_generation_prompt=True, tokenize=False)
    input_ids = tokenizer.encode(
        formatted_prompt,
        add_special_tokens=False,
        return_tensors="pt"
    )

    # Set seed for reproducibility
    seed = 42
    torch.manual_seed(seed)

    # ✋Hparamsは、モデルに合わせて調整（HuggingFace のモデルカード参照）
    tokens = model.generate(
        input_ids.to(device=model.device),
        max_new_tokens=1024,
        temperature=0.5,
        repetition_penalty=1.15,
        top_p=0.95,
        do_sample=True,
    )

    # Remove the input tokens from the generated tokens before decoding
    output_tokens = tokens[0][len(input_ids[0]):]
    return tokenizer.decode(output_tokens, skip_special_tokens=True)

def process_dataset(input_file, output_file):
    # Read dataset and generate predictions
    with jsonlines.open(input_file) as reader, jsonlines.open(output_file, mode='w') as writer:
        for obj in reader:
            prompt = obj['input_text']
            generated_text = generate_text(prompt)
            writer.write({"pred": generated_text})

            print("==============================")
            print(f"Q. {prompt}")
            print(f"A. {generated_text}")
            print()

# Process the dataset
input_dataset = '/content/dataset.jsonl'
output_predictions = '/content/preds.jsonl'
process_dataset(input_dataset, output_predictions)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Q. 仕事の熱意を取り戻すためのアイデアを5つ挙げてください。
A. 1. 目標を設定する: 仕事に対する情熱を取り戻したい場合、まず自分のキャリアのビジョンと現在のポジションとのギャップを特定することが重要です。また、自分自身の価値観や興味に沿った仕事に就くことでやりがいを感じることができるかもしれません。

2. 新しいスキルを学ぶ: 新たなスキルを身につけることで、職場での役割を再定義し、新たな挑戦に備えることができます。また、特定の分野の知識を深めることも、モチベーションの向上につながります。

3. キャリアカウンセラーに相談する: キャリアカウンセリングを受けることで、今の状況を客観的に分析し、その後の計画を立てることができます。また、キャリアカウンセラーは、あなたの強みと弱み、キャリア上の目標、将来の夢について話し合うことができます。

4. 自己啓発プログラムに参加する: 自己啓発プログラムやワークショップに参加することで、自信を高め、成功への意欲を高めることができます。これらのプログラムでは、コミュニケーション能力、リーダーシップ、問題解決などのスキルを学びます。

5. プロジェクトチームに参加する: 他の同僚と共同で作業に取り組めば、より創造的な方法で仕事を進めることができ、共通の目標に向かって協力しやすくなります。



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Q. 花粉症の方々にとって、春は憂鬱な季節です。しかし、春には素敵な花や新緑もたくさんあります。そこで、花粉症でも春を楽しむコツを3つ考えてください。
A. 1. 花粉の飛散量が少ない品種を選ぶ: 花粉症の人は、花粉の少ない植物を選びましょう。例えば、桜の木、竹、バラなどの花は花粉が少ないのでおすすめです。また、開花時期も短く、花粉が飛ばない期間が長いため、花粉症の症状が出にくい植物もあります。ただし、これらの植物は花粉の量が少なくても、アレルギー反応を引き起こす可能性のある化学物質を含んでいます。そのため、アレルギーのある人は注意が必要です。

2. 花粉シーズン前に庭の手入れをする: 花粉シーズンが始まる前に、庭の草木に害虫を寄せ付けないようにしましょう。花粉の放出を促さないために、特に春先に殺虫剤を使用することは避けましょう。これにより、花粉の量が増える可能性を減らし、あなたの花粉症の症状が軽くなるかもしれません。

3. マスクを着用する: マスクとメガネは、花粉の季節を乗り切るための必需品の1つです。この2つのアイテムは、鼻から侵入したアレルゲンを吸収して肺に到達するのを防ぐことで、花粉症の症状を軽減するのに役立ちます。さらに、花粉は空気中を自由に飛び回っているので、外出時は常にマスクを着用しておくのがベストです。



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Q. 漢字に対応する意味を選び、対応する数字とアルファベットのペアを解答してください。

1. 立派　A. すばらしい
2. 開拓　B. ある事柄について知っている内容。
3. 知識　C. 物事に味わいや価値を加える
4. 賞賛　D. 新しい領域を切り開く
5. 味付け　E. 高い評価を与える
A. 1. A.素晴らしい 978-0-520-26675-8
2. B. 知見 978-1-4234-9448-2
3. C. 洞察 978-0-06-252769-2
4. D. 革新性 978-1-5918-4657-7
5. E. 評価 978-0-06-252767-5



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Q. 次の物語の展開を予想してみましょう。

ある日、主人公のもとに不思議な手紙が届きました。手紙には「今夜の満月に、森の奥深くにある洞窟に来てください。あなたを待っています。」と書かれていました。主人公はその手紙に従い、夜中に洞窟の入り口にたどり着きました。中に入ると、謎めいた人物と出会い……。
A. 主人公は洞窟の奥深くへと進んでいき、やがて幽霊のような存在が現れました。その人物は、古い物語や伝説を語るためにここに集まった「伝説の守護者」でした。彼らは主人公に「この地を訪れる人々を救い、森を守ること」を命じます。主人公は自分の運命を受け入れ、冒険に乗り出すことにしました。

旅を続けるうちに、主人公たちは様々な障害に直面します。中には危険な生き物や、隠された宝物など、新しい発見もあります。しかし、最も困難な課題の1つは、キャラクターが自分の過去の行動や選択によって引き起こされる恐ろしい結果に対処することです。この過程で、主人公が自分自身の真実を認識し、成長していきます。

そして、主人公たちが洞窟から脱出するとき、彼らは「伝説の守護者」が彼らに与え続けた任務を達成することに成功していました。それ以来、森の人々は平和で繁栄するようになり、かつてないほどの繁栄を見せます。

最後に、主人公の活躍により、森を守るという使命を果たすために、他のキャラクターも力を合わせて協力し始めます。彼らは一緒に仕事をし、彼らの絆を強め、友情を築くことで、自分たちのコミュニティをより強固なものにしています。

Q. 次の文は1つの物語の文の順番をシャッフルしたものです。適切な順に並び替えてください。

1. 「果物だけではなく、人生も同じです。協力しなくては、全てをこなすことはできません。」
2. 老人は、広い畑で色々な果物を育てていました。
3. ある日、孫が老人に『こんなに広い畑で、全ての果物を一人で育てるのは大変そうですね』と尋ねました。
4. 昔、あるところに老人と孫がいました。
5. 老人は微笑みながら次のように答えました。

A. 1. 昔、あるところに老人と孫がいました。
2. その老人は、大きな畑に様々な種類の果物が植わっていました。
3. ある日、孫は老人にこう言いました、「この広大な畑の中で、すべての果物を一人でするのは大変そうですね。」
4. 老人は笑顔でこう答え

# 🛑 STOP!

**`preds.jsonl` をローカルストレージにダウンロード**する。

必要に応じて、[GPT-4 による評価](https://github.com/Northern-System-Service/gpt4-autoeval) を実行する。